In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from torchvision import datasets
from tqdm import tqdm

import os
import time

# GPU 설정, 없으면 CPU 를 사용하는 코드
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 데이터가 저장된 경로
data_dir = r'D:\Data\dAiv\train'

# 이미지 전처리 (수정 가능)
image_transforms = {
    # 훈련 데이터 증강
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    # 검증 데이터에서는 증강하지 않습니다
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

# 해당 데이터셋이 있는 경로로부터 (훈련,검증) 데이터를 불러옵니다
data = {
    'train':
    datasets.ImageFolder(root=data_dir, transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root=data_dir, transform=image_transforms['valid']),
}

# DataLoader 선언, batch_size 는 64 , shuffle 은 true 로 설정했습니다. (배치사이즈 수정 가능)
dataloaders = {
    'train': DataLoader(data['train'], batch_size=64, shuffle=True),
    'valid': DataLoader(data['valid'], batch_size=64, shuffle=True)
}

# 사전학습 모델을 불러옵니다. 사전학습된 resnet 의 경우에는 1000개 클래스 분류이므로 출력 노드가 1000개입니다.
# 하지만 우리가 쓸 데이터는 클래스가 33개이므로, 출력층 부분에 전연결층을 추가해서 클래스 갯수에 맞게 노드 갯수를 수정할 수 있습니다.
# 모델 변경은 자유입니다!
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataloaders['train'].dataset.classes))

model = model.to(device)

# 옵티마이저와 손실함수. CCE (CrossEntropy 손실함수를 사용), 옵티마이저는 Adam.
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


c:\Users\User\Anaconda3\envs\MachineLearning\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\Anaconda3\envs\MachineLearning\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\User/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 27.8MB/s]


In [2]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

    return model

# train the model
trained_model = train_model(model, loss_func, optimizer, num_epochs=25)

# save the trained model
torch.save(trained_model.state_dict(), 'model_weights.pth')


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 0/24
----------
train Loss: 0.1216 Acc: 0.9678


  4%|▍         | 1/25 [03:32<1:24:48, 212.00s/it]

valid Loss: 0.3704 Acc: 0.8912
Epoch 1/24
----------
train Loss: 0.0290 Acc: 0.9925


  8%|▊         | 2/25 [05:55<1:05:47, 171.65s/it]

...

valid Loss: 0.0000 Acc: 1.0000
Epoch 23/24
----------
train Loss: 0.0054 Acc: 0.9986


 96%|█████████▌| 24/25 [1:08:57<03:00, 180.64s/it]

valid Loss: 0.0006 Acc: 0.9998
Epoch 24/24
----------
train Loss: 0.0022 Acc: 0.9993


100%|██████████| 25/25 [1:11:59<00:00, 172.80s/it]

valid Loss: 0.0000 Acc: 1.0000
